In [1]:
import pandas as pd
from coypu.utils.query import Query
from coypu.utils.credentials import *
from query.queries import *
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "browser"
import matplotlib.pyplot as plt
import numpy as np

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

coypu.utils


In [ ]:
cmemc_query_imp = Query(client_url_imp, client_id_imp, client_secret_imp , 'oauth')
    
fuseki_infai = Query(fuseki_endpoint_infai, fuseki_user_infai, fuseki_pw_infai, 'basic')

fdquery = Query("http://localhost:5002/sparql", is_fdq=True)

In [ ]:
print(query_2_fdq_desc)
print(query_2_fdq)
print('########################### EXAMPLE ##########################\n{}'.format(query_2_fdq_ex))

fdquery.post_query(query_2_fdq_ex)
fdquery.to_save('../../query_results/', filename='query_2_fdq_ex')

In [3]:
from sqlalchemy import create_engine

engine = create_engine(
        "mysql+pymysql://{user}:{pw}@{hostname}:{portno}/{db}".format(
            hostname=host, db=dbname, user=uname, pw=pwd, portno=port
        ),)

df = pd.read_csv('../../query_results/query_2_fdq_ex.csv', encoding='utf-8')
df.rename(columns=lambda x: x.replace('.value', ''), inplace=True)
try:
    df.to_sql('query_2_fdq_ex', con=engine, if_exists='fail', index=False)
except Exception as e:
    print('Exeception: {}',e)

Exeception: {} Table 'query_2_fdq_ex' already exists.


In [4]:
from sqlalchemy import create_engine

engine = create_engine(
        "mysql+pymysql://{user}:{pw}@{hostname}:{portno}/{db}".format(
            hostname=host, db=dbname, user=uname, pw=pwd, portno=port))

df = pd.read_sql(query_2_fdq_ex_sql, con=engine)

df.to_csv('../../query_results/query_2_fdq_ex_sql.csv', encoding='utf-8', index=False)

In [2]:
df = pd.read_csv('../../query_results/query_2_fdq_ex_sql.csv', encoding='utf-8')

df['WikiData-WB(AvgLifeExpectancy)'] = np.abs(df['year_avg_exp_wiki']-df['year_avg_exp_WB'])
df.head(10)

,country_code,country_name,year_WB,year_avg_exp_wiki,year_avg_exp_WB,WikiData-WB(AvgLifeExpectancy)
0,IDN,Indonesia,1974,54.88300,54.8430,0.04000
1,IDN,Indonesia,1981,58.51550,58.4890,0.02650
2,TKM,Turkmenistan,2010,66.65700,66.6570,0.00000
3,TLS,Timor-Leste,2010,67.31300,67.1860,0.12700
4,TTO,Trinidad and Tobago,2010,69.75600,71.9460,2.19000
5,UGA,Uganda,2010,57.15300,57.0990,0.05400
6,UKR,Ukraine,2010,70.26537,70.2654,0.00003
7,UZB,Uzbekistan,2010,70.00500,69.6720,0.33300
8,VUT,Vanuatu,2010,70.72100,69.1230,1.59800
9,ZAF,South Africa,2010,55.88800,57.6690,1.78100


In [3]:
import plotly.express as px
fig = px.choropleth(df[df['year_WB']==2016], locations="country_code",
                    color="WikiData-WB(AvgLifeExpectancy)", # lifeExp is a column of gapminder
                    hover_name="country_name", # column to add to hover information
                    color_continuous_scale=px.colors.sequential.Magenta)
fig.write_html('../../query_results/query_2_fdq_ex_sql.html')
fig.show()